# ![Untitled (1).png](<attachment:Untitled (1).png>)

In [117]:
import streamlit as st 
import pdfplumber
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
# import nltk
# from rank_bm25 import BM25Okapi
# import numpy as np

In [118]:
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        with pdfplumber.open(pdf) as pdf:
            for page in pdf.pages:
                text += page.extract_text()
    return text

In [119]:
pdf_path = (
    "C:/Users/Admin/Desktop/MY/INTERNSHIPS/GovConn.ai/Info/Research_Paper_ChatPDF.pdf"
)
raw_text = get_pdf_text([pdf_path])

In [120]:
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n", chunk_size=1000, chunk_overlap=200, length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [121]:
chunks = get_text_chunks(raw_text)
print(chunks)

['Cite as: Panda, S. (2023). Enhancing PDF interaction for a more engaging user experience in library:\nIntroducing ChatPDF. IP Indian Journal of Library Science and Information Technology, 8(1), 20–25.\nhttps://doi.org/10.18231/j.ijlsit.2023.004\nEnhancing PDF Interaction for a More Engaging User Experience in\nLibrary: Introducing ChatPDF\nAbstract\nPurpose: The paper introduces ChatPDF as a solution to the limited interaction capabilities\nof traditional PDF readers in library systems. It highlights the benefits of ChatPDF for\nenhancing user engagement and satisfaction with digital resources in libraries, and identifies\npotential areas for implementationand opportunitiesfor further research.\nDesign/methodology/approach: The paper addresses the issue of limited PDF interaction\ncapabilities of traditional PDF readers in library systems and their potential to cause\nfrustration and disengagement among users. It proposes ChatPDF, an online software', 'capabilities of traditional PDF

In [122]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [123]:
# # Creating objects representing each document with both text and embeddings
# class Document:
#     def __init__(self, text, score, embeddings, metadata=None):
#         self.page_content = text
#         self.embeddings = embeddings
#         self.metadata = {"score": score} if metadata is None else metadata

In [124]:
# Creating objects representing each document with both text and embeddings
class Document:
    def __init__(self, text, embeddings, metadata=None):
        self.page_content = text
        self.embeddings = embeddings
        self.metadata = metadata

In [125]:
documents = []
for chunk in chunks:
    embedding_vectors = embeddings.embed_documents([chunk])
    # print(embedding_vectors)
    # Creating document object for each chunk
    document = Document(chunk, embedding_vectors[0], metadata={})
    documents.append(document)

print(documents)

[<__main__.Document object at 0x0000018CBF38C450>, <__main__.Document object at 0x0000018C9BF973D0>, <__main__.Document object at 0x0000018CA3285DD0>, <__main__.Document object at 0x0000018CBF3EA250>, <__main__.Document object at 0x0000018CC032CB90>, <__main__.Document object at 0x0000018CA2800490>, <__main__.Document object at 0x0000018CBF507010>, <__main__.Document object at 0x0000018CBF3AE110>, <__main__.Document object at 0x0000018C8FEC2D10>, <__main__.Document object at 0x0000018CBDCD9350>, <__main__.Document object at 0x0000018CC0173F10>, <__main__.Document object at 0x0000018CA2829DD0>, <__main__.Document object at 0x0000018C9FC8D810>, <__main__.Document object at 0x0000018C9BFA1DD0>, <__main__.Document object at 0x0000018CA533DA90>, <__main__.Document object at 0x0000018CA1AAA650>, <__main__.Document object at 0x0000018CBF3B8550>, <__main__.Document object at 0x0000018CBFFE8250>, <__main__.Document object at 0x0000018CC01E7FD0>, <__main__.Document object at 0x0000018CBDCE6550>,

In [126]:
# documents=[]
# for chunk in chunks:
#     embedding_vectors= embeddings.embed_documents([chunk])
#     # print(embedding_vectors)
#     doc_score=1.0 #currently setting it to 1.0
#     # Creating document object for each chunk
#     document=Document(chunk, embedding_vectors[0], doc_score)
#     documents.append(document)

# print(documents)

In [127]:
# @classmethod
# def from_documents(cls, documents, embedding, **kwargs):
#     texts = [d.page_content for d in documents]
#     return cls.from_texts(texts, embedding, **kwargs)

In [128]:
db = FAISS.from_documents(documents, embeddings)

In [129]:
# FAISS(Facebook AI Similarity Search)
query="what is this pdf about?"
# query_chunks= get_text_chunks(query)
# print(query_chunks)

In [130]:
# document2=[]
# for query_chunk in query_chunks:
#     query_embeddings = embeddings.embed_documents([query_chunk])

#     query_document=Document(query_chunk, query_embeddings[0], metadata={})
#     document2.append(query_document)
# print(document2)
query_embeddings=embeddings.embed_documents([query])
print(query_embeddings)

[[-0.08536584675312042, 0.09150689095258713, -0.08419298380613327, 0.02204255573451519, 0.022216223180294037, 0.018777862191200256, 0.03154744952917099, 0.002281725639477372, -0.10130906850099564, 0.0036651636473834515, 0.04448528587818146, 0.09752530604600906, 0.012333959341049194, -0.06200604885816574, -0.15755796432495117, 0.005351174157112837, -0.03027423471212387, -0.11461827158927917, -0.03213956579566002, 0.07018972933292389, -0.0033783921971917152, 0.007904857397079468, 0.03385080769658089, 0.013757845386862755, -0.00012495237751863897, 0.0022582539822906256, -0.053301602602005005, -0.011961321346461773, -0.010334695689380169, -0.023633478209376335, 0.05988442525267601, 0.0803077295422554, 0.03502393886446953, 0.007845132611691952, 0.02264735847711563, -0.002206980949267745, 0.05608081445097923, -0.013835192658007145, 0.07526884227991104, 0.038035959005355835, 0.014889180660247803, -0.12405292689800262, -0.008694722317159176, 0.01683998294174671, -0.0233067087829113, 0.04453589

In [131]:
docs = db.similarity_search(query)
# print(docs)
content = "\n".join([d.page_content for d in docs])
# print(content)

In [132]:
# ranked_docs = sorted(docs, key=lambda x: x[1], reverse=True)

In [133]:
# print("Top 5 most relevant documents:")
# for doc in docs:
#     print(
#         f"Relevance score:{doc.metadata['score']:.2f}"
#     )  # assessing score key in metadata dictionary and :.2f formats the float value to have two decimal places.
#     print(doc.page_content) #prints text chunk from pdf file
#     print("-" * 5)

In [134]:
qa_prompt = "Use the following pieces of context to answer the user's question. If you don't know the answer, just say that you don't know, don't try to make up an answer.----------------"
input_text = qa_prompt + "\nContext:" + content + "\nUser question:\n" + query
print(input_text)

Use the following pieces of context to answer the user's question. If you don't know the answer, just say that you don't know, don't try to make up an answer.----------------
Context:experience and hinderengagement.
i) Limited interactivity: Traditional PDFs are static documents that offer limited
interactivity, which can make them less engaging and less useful for users who are
looking for more dynamic content. For example, users may not beable to interact with or
manipulate the information in a traditional PDF in the same way that they could with a
webpage orother digital content.
ii) Inefficient navigation: PDFs can be challenging to navigate, particularly for longer
documents. Users may have difficulty finding specific information or moving between
different sections of thedocument, which can lead tofrustration and reduced engagement.
Electronic copy available at: https://ssrn.com/abstract=4616560iii) Limited accessibility: Traditional PDFs may not be accessible to all users, parti

In [135]:
# load_dotenv()
# api_key= os.getenv("ChatAnthropic_API")
# llm = ChatAnthropic(
#     api_key=api_key, model_name="claude-3-sonnet-20240229"
# )

In [136]:
# result = llm.invoke(input_text)
# print(result.content)